In [67]:
import pandas as pd

txns = pd.read_excel('profit-test.xlsx')
txns['Date'] = pd.to_datetime(txns['Date'], dayfirst=True, infer_datetime_format=True)

In [68]:
txns['Profit'] = 0

In [86]:
token = 'ADA'
tp1 = 'trade-buy'
tp2 = 'buy-in'
ada_buys = txns[(txns['Token'] == token) & ((txns['Type'] == tp1) | (txns['Type'] == tp2))].copy()

In [87]:
method = 'HIFO'
if method == 'FIFO':
    ada_buys = ada_buys.sort_values(by='Date', ascending=False).copy()
elif method == 'HIFO':
    ada_buys = ada_buys.sort_values(by='Basis', ascending=True).copy()

In [88]:
tp1 = 'trade-sell'
tp2 = 'sell-out'
ada_sells = txns[(txns['Token'] == token) & ((txns['Type'] == tp1) | (txns['Type'] == tp2))].copy()
ada_sells = ada_sells.sort_values(by='Date', ascending=False).copy()

In [89]:
ada_buys

,Date,Wallet,Token,Amount,Basis,Value,Fee,Type,Profit
17,2021-12-10,Coinbase Pro,ADA,150.00,1.256267,188.440000,0.9400,buy-in,0
2,2021-07-29,Coinbase Pro,ADA,75.00,1.266267,94.970000,0.4700,buy-in,0
1,2021-07-12,Coinbase Pro,ADA,75.00,1.306533,97.990000,0.4900,buy-in,0
16,2021-11-26,Coinbase Pro,ADA,125.00,1.555440,194.430000,0.6800,buy-in,0
15,2021-11-24,Coinbase Pro,ADA,8.03,1.605230,12.890000,0.0400,buy-in,0
14,2021-11-24,Coinbase Pro,ADA,191.97,1.605615,308.230000,1.0800,buy-in,0
3,2021-08-18,Coinbase Pro,ADA,30.00,1.909333,57.280000,0.2800,buy-in,0
13,2021-10-27,Coinbase Pro,ADA,100.00,2.010000,201.000000,1.0000,buy-in,0
0,2021-05-17,Coinbase Pro,ADA,50.00,2.010000,100.500000,0.5000,buy-in,0
6,2021-10-13,Coinbase Pro,ADA,50.00,2.110600,105.530000,0.5300,buy-in,0


In [114]:
ada_sells

,Date,Wallet,Token,Amount,Basis,Value,Fee,Type,Profit
20,2022-02-24,Coinbase Pro,ADA,1000.0,1.148685,870.560963,0.0,trade-sell,0
18,2022-01-23,Coinbase Pro,ADA,200.0,0.932915,214.381878,0.0,trade-sell,0
19,2022-01-23,Coinbase Pro,ADA,200.0,0.932915,214.381878,0.0,trade-sell,0


In [179]:
selldict = ada_sells.to_dict('index')
buydict = ada_buys.to_dict('index')

In [161]:
   
pnl = {}
tolerance = 1e-10
while selldict:
    sell = selldict.popitem()
    amount = sell[1]['Amount']
    token = sell[1]['Token']
    while amount > tolerance:
        print(f"Sell Lot: [{sell[0]}] {sell[1]['Amount']} {token}.")
        buy = buydict.popitem()
        print(f"From Lot: [{buy[0]}] {buy[1]['Amount']} {token}.")
        leftover = amount - buy[1]['Amount']
        if leftover >= 0:
            profit = buy[1]['Amount'] * (sell[1]['Basis'] - buy[1]['Basis'])
            amount = leftover
        else:
            profit = (buy[1]['Amount']+leftover) * (sell[1]['Basis'] - buy[1]['Basis'])
            buy[1]['Amount'] = -leftover
            amount = 0
            buydict[buy[0]] = buy[1]
            pnl[sell[0]] = sell[1]
        sell[1]['Profit'] += profit
            

To Sell: 19 200.0
Sell Lot: 4 50.0
To Sell: 19 200.0
Sell Lot: 5 100.0
To Sell: 19 200.0
Sell Lot: 10 100.0
To Sell: 18 200.0
Sell Lot: 10 50.0
To Sell: 18 200.0
Sell Lot: 12 250.0
To Sell: 20 1000.0
Sell Lot: 12 100.0
To Sell: 20 1000.0
Sell Lot: 7 100.0
To Sell: 20 1000.0
Sell Lot: 8 100.0
To Sell: 20 1000.0
Sell Lot: 9 100.0
To Sell: 20 1000.0
Sell Lot: 11 100.0
To Sell: 20 1000.0
Sell Lot: 6 50.0
To Sell: 20 1000.0
Sell Lot: 0 50.0
To Sell: 20 1000.0
Sell Lot: 13 100.0
To Sell: 20 1000.0
Sell Lot: 3 30.0
To Sell: 20 1000.0
Sell Lot: 14 191.97
To Sell: 20 1000.0
Sell Lot: 15 8.03
To Sell: 20 1000.0
Sell Lot: 16 125.0


In [162]:
pnl

{19: {'Date': Timestamp('2022-01-23 00:00:00'),
  'Wallet': 'Coinbase Pro',
  'Token': 'ADA',
  'Amount': 200.0,
  'Basis': 0.9329146762648908,
  'Value': 214.381877666176,
  'Fee': 0.0,
  'Type': 'trade-sell',
  'Profit': -282.7840263684993},
 18: {'Date': Timestamp('2022-01-23 00:00:00'),
  'Wallet': 'Coinbase Pro',
  'Token': 'ADA',
  'Amount': 200.0,
  'Basis': 0.9329146762648908,
  'Value': 214.381877666176,
  'Fee': 0.0,
  'Type': 'trade-sell',
  'Profit': -251.1729249817803},
 20: {'Date': Timestamp('2022-02-24 00:00:00'),
  'Wallet': 'Coinbase Pro',
  'Token': 'ADA',
  'Amount': 1000.0,
  'Basis': 1.148684632721107,
  'Value': 870.560962960835,
  'Fee': 0.0,
  'Type': 'trade-sell',
  'Profit': -829.1211522816157}}

In [163]:
df = pd.DataFrame.from_dict(pnl, orient='index')
df

,Date,Wallet,Token,Amount,Basis,Value,Fee,Type,Profit
19,2022-01-23,Coinbase Pro,ADA,200.0,0.932915,214.381878,0.0,trade-sell,-282.784026
18,2022-01-23,Coinbase Pro,ADA,200.0,0.932915,214.381878,0.0,trade-sell,-251.172925
20,2022-02-24,Coinbase Pro,ADA,1000.0,1.148685,870.560963,0.0,trade-sell,-829.121152


In [165]:
list(set(df['Token'].unique()))

['ADA']

In [17]:
import pandas as pd
def pnl(txns, method):
    txns['Date'] = pd.to_datetime(txns['Date'], dayfirst=True, infer_datetime_format=True)
    txns['Profit'] = 0
    
    df = pd.DataFrame()
    tolerance = 1e-10
    i = 0
    for token in list(set(txns['Token'].unique())):
        print(token)
        tk = txns[(txns['Token'] == token)].copy()
        tp1 = 'trade-buy'
        tp2 = 'buy-in'
        buys = tk[((txns['Type'] == tp1) | (tk['Type'] == tp2))].copy() 

        if method == 'FIFO':
            buys = buys.sort_values(by='Date', ascending=False)
        elif method == 'HIFO':
            buys = buys.sort_values(by='Basis', ascending=True)

        tp1 = 'trade-sell'
        tp2 = 'sell-out'
        sells = tk[((tk['Type'] == tp1) | (tk['Type'] == tp2))].copy()
        sells = sells.sort_values(by='Date', ascending=False)

        selldict = sells.to_dict('index')
        buydict = buys.to_dict('index')

        calcs = {}
        while selldict:
            sell = selldict.popitem()
            date = sell[1]['Date']
            wallet = sell[1]['Wallet']
            amount = sell[1]['Amount']
            
            while amount > tolerance:
                try:
                    buy = buydict.popitem()
                    leftover = amount - buy[1]['Amount']
                    calcs[i] = {'Date':date, 'Wallet':wallet, 'Token':token}
                    calcs[i]['Sell Basis'] = sell[1]['Basis']
                    calcs[i]['Buy Basis'] = buy[1]['Basis']
                    calcs[i]['Period'] = sell[1]['Date'] - buy[1]['Date']
                    if leftover >= 0:
                        calcs[i]['Amount'] = buy[1]['Amount']
                        profit = calcs[i]['Amount'] * (calcs[i]['Sell Basis'] - calcs[i]['Buy Basis'])
                        calcs[i]['Profit'] = profit
                        amount = leftover
                    else:     
                        calcs[i]['Amount'] = buy[1]['Amount']+leftover
                        profit = calcs[i]['Amount'] * (calcs[i]['Sell Basis'] - calcs[i]['Buy Basis'])
                        calcs[i]['Profit'] = profit
                        buy[1]['Amount'] = -leftover
                        buydict[buy[0]] = buy[1]
                        amount = 0
                    df = pd.concat([df, pd.DataFrame.from_dict(calcs, orient='index')])
                except KeyError:
                    break
    
    df.reset_index(inplace=True)
    df.drop(columns=['index'], inplace=True)
    return df

In [18]:
data = pd.read_csv('transactions.csv')
result = pnl(data, 'FIFO')
result

USDC


C:\Users\jko04\AppData\Local\Temp\ipykernel_7544\3817991118.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  buys = tk[((txns['Type'] == tp1) | (tk['Type'] == tp2))].copy()
C:\Users\jko04\AppData\Local\Temp\ipykernel_7544\3817991118.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  buys = tk[((txns['Type'] == tp1) | (tk['Type'] == tp2))].copy()
C:\Users\jko04\AppData\Local\Temp\ipykernel_7544\3817991118.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  buys = tk[((txns['Type'] == tp1) | (tk['Type'] == tp2))].copy()
C:\Users\jko04\AppData\Local\Temp\ipykernel_7544\3817991118.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  buys = tk[((txns['Type'] == tp1) | (tk['Type'] == tp2))].copy()
C:\Users\jko04\AppData\Local\Temp\ipykernel_7544\3817991118.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  buys = tk[((txns['T

FTM
UNI
AVAX
LINK
USDT
WBTC
LTC
ATOM
ADA


C:\Users\jko04\AppData\Local\Temp\ipykernel_7544\3817991118.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  buys = tk[((txns['Type'] == tp1) | (tk['Type'] == tp2))].copy()
C:\Users\jko04\AppData\Local\Temp\ipykernel_7544\3817991118.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  buys = tk[((txns['Type'] == tp1) | (tk['Type'] == tp2))].copy()
C:\Users\jko04\AppData\Local\Temp\ipykernel_7544\3817991118.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  buys = tk[((txns['Type'] == tp1) | (tk['Type'] == tp2))].copy()
C:\Users\jko04\AppData\Local\Temp\ipykernel_7544\3817991118.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  buys = tk[((txns['Type'] == tp1) | (tk['Type'] == tp2))].copy()


XLM
COMP
BTC
LRC


C:\Users\jko04\AppData\Local\Temp\ipykernel_7544\3817991118.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  buys = tk[((txns['Type'] == tp1) | (tk['Type'] == tp2))].copy()
C:\Users\jko04\AppData\Local\Temp\ipykernel_7544\3817991118.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  buys = tk[((txns['Type'] == tp1) | (tk['Type'] == tp2))].copy()
C:\Users\jko04\AppData\Local\Temp\ipykernel_7544\3817991118.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  buys = tk[((txns['Type'] == tp1) | (tk['Type'] == tp2))].copy()
C:\Users\jko04\AppData\Local\Temp\ipykernel_7544\3817991118.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  buys = tk[((txns['Type'] == tp1) | (tk['Type'] == tp2))].copy()
C:\Users\jko04\AppData\Local\Temp\ipykernel_7544\3817991118.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  buys = tk[((txns['T

CRO
BTRFLY
POWR
MIOTA
DOT
DOGE
QTUM
ETH
BAT
USD


C:\Users\jko04\AppData\Local\Temp\ipykernel_7544\3817991118.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  buys = tk[((txns['Type'] == tp1) | (tk['Type'] == tp2))].copy()
C:\Users\jko04\AppData\Local\Temp\ipykernel_7544\3817991118.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  buys = tk[((txns['Type'] == tp1) | (tk['Type'] == tp2))].copy()


,Date,Wallet,Token,Sell Basis,Buy Basis,Period,Amount,Profit
0,2021-05-04,Coinbase Pro,USDC,55775.977150,1.000000,0 days,111.555000,6.221978e+06
1,2021-05-12,Coinbase,USDC,59494.308600,1.000000,8 days,50.000000,2.974665e+06
2,2021-05-19,Coinbase Pro,USDC,1.000000,1.000000,15 days,476.890000,0.000000e+00
3,2021-10-02,Coinbase Pro,USDC,2.265811,1.000000,151 days,226.125000,2.862315e+02
4,2021-10-04,Coinbase Pro,USDC,1.000000,1.000000,153 days,500.000000,0.000000e+00
...,...,...,...,...,...,...,...,...
150,2021-05-17,Coinbase Pro,ETH,3400.000000,209.997427,921 days,0.100000,3.190003e+02
151,2021-05-18,Coinbase Pro,ETH,3500.000000,209.997427,922 days,0.100000,3.290003e+02
152,2021-11-18,Coinbase Pro,ETH,4056.808777,209.997427,1106 days,0.049056,1.887084e+02
153,2021-11-18,Coinbase Pro,ETH,4058.096482,209.997427,1106 days,0.001294,4.978324e+00


In [16]:
result.to_csv('pnl-2021.csv')